<br>

<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: #34558b; background-color: #ffffff;">BirdCLEF '22 - Visualize Predictions</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

<br>

---

<br>

<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">👏 &nbsp; IF YOU FORK THIS OR FIND THIS HELPFUL &nbsp; 👏</b><br><br><b style="font-size: 22px; color: darkorange">PLEASE UPVOTE!</b><br><br>This was a lot of work for me and while it may seem silly, it makes me feel appreciated when others like my work. 😅
</div></center>

---

<br>

<a id="imports"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: #34558b;" id="imports">0&nbsp;&nbsp;IMPORTS&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

**You don't need all of these... it's just easier for me to copy and paste them all over from my other notebooks**

In [ ]:
print("\n... IMPORTS STARTING ...\n")

# print("\n... PIP/APT INSTALLS AND DOWNLOADS/ZIP STARTING ...")
# print("... PIP/APT INSTALLS COMPLETE ...\n")

print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import tensorflow_io as tfio; print(f"\t\t– TENSORFLOW I/O VERSION: {tfio.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None; pd.set_option('display.max_columns', 50);
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from pandarallel import pandarallel; pandarallel.initialize()
from sklearn.model_selection import GroupKFold, StratifiedKFold

# Competition Specific (AUDIO)
import librosa; import librosa.display
import soundfile as sf
from  soundfile import SoundFile
import IPython.display as ipd

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import hashlib
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import json
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
from joblib import Parallel, delayed
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image, ImageEnhance
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2

import plotly.io as pio
print(pio.renderers)

def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")

<br>

<a id="background_information"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #34558b; background-color: #ffffff;" id="setup">1&nbsp;&nbsp;SETUP&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---

Please note that your setup may differ based on your particular modelling pipeline. **The setup should not impact the visualization tool as it only digests your predictions.** I took this as an opportunity to distil my pipeline and explore it hence the detailed explanation.

**In other words, you could skip this cell if you already had a dataframe that looks like the `sample_submission` dataframe in terms of format but contains valid file information. The visualization tool will take either the `boolean` version of this (True v. False) or a logit version of this (0.00-1.00).**

***Example 1 - Bools***
```
row_id,target
soundscape_1000170626_akiapo_5,False
soundscape_1000170626_akiapo_10,False
soundscape_1000170626_akiapo_15,False
etc.
```

***Example 2 - Probabilities***
```
row_id,target
soundscape_1000170626_akiapo_5,0.01
soundscape_1000170626_akiapo_10,0.025
soundscape_1000170626_akiapo_15,0.05
etc.
```

<br>

---

<br>

The setup will take **5-10 minutes** and is composed of the following steps:

1. Add file path to dataframe
2. Add audio information (duration) to dataframe
3. Remove all audio clips that don't fit within a given duration length *(optional)*
  * This is to help us capture a good sampling of Hawaiian birds along with other birds
  * This also helps us avoid ending up with very-short or very-long clips of audio
  * For reference my extents are:
    * Minimum: 3 Seconds
    * Maximum: 180 seconds
    * We also take a very particular clip that has a duration of 249.048 as this is the only clip that contains a particular Hawaiian bird.
4. Reduce the size of the dataframe considerably by sampling audio clips for each respective bird
  * Note that we give a preference for audio clips that are close to 60 seconds (as this is the length of the test audio clips
  * Note that we pad/clip audio clips to be exactly 60 seconds to mimic the hidden test data
  * Note that the **`duration_goal`** parameter is how we control the degree of reduction (i.e. 66,000 is 1/10th the size of the expected test set and is a decent starting point)
5. Break the clips down into 7 second segments with a 2 second step (i.e. a 60 second clip will be made of 28 7-second segments) and save these segments as Mel Spectrograms to be used for inferencing
6. Find weights that will allow us to map the 7-second segments onto the 5-second scoring segments
7. Conduct inference and create the prediction dataframe using nocall and multilabel models.
  * <b><font color="red">NOTE: I have not made these models public yet, so this pipeline won't work if you just fork and run it. It will however, work for YOUR predictions as mentioned previously. I will also most likely make my models public in the near future.</font></b>

In [ ]:
#################################
#####  CONSTANTS AND KWARGS  ####
#################################
DATA_DIR = "/kaggle/input/birdclef-2022"
TMP_OUT_DIR = "/tmp/data_dir"
SR = 32_000 # Define the Sample Rate
WINDOW = 7 # Define the Window Size (in seconds)
WINDOW_SR = WINDOW*SR # Define the Window Size (in samples)
STEP = 2 # Define the Step/Overlap Size (in seconds)
OVERLAP = WINDOW-STEP # Define the Step/Overlap Size (in seconds)
STEP_SR = int(round(STEP*SR)) # Define the Step/Overlap Size (in samples)
OVERLAP_SR = int(round(OVERLAP*SR)) # Define the Step/Overlap Size (in samples)
DISCARD = 10 # Define the Discard Threshold Time (in seconds)(impossible for inference)
DISCARD_SR = int(DISCARD*SR) # Define the Discard Threshold Time (in samples)
NFFT = SR//10
HOP_LENGTH = SR//40
WIN_LENGTH = NFFT
N_MELS = 128
MEL_LEN = 281
F_MIN = 0
F_MAX = SR//2
MAX_CLIP_LEN = 60
MAX_CLIP_LEN_SR = MAX_CLIP_LEN*SR

os.makedirs(TMP_OUT_DIR, exist_ok=True)
melspec_kwargs = dict(
    sr=SR, 
    n_mels=N_MELS, 
    fmin=F_MIN, 
    fmax=F_MAX, 
    n_fft=NFFT, 
    hop_length=HOP_LENGTH,
)
segment_kwargs = dict(
    sample_rate=SR, 
    window_size=WINDOW_SR, 
    step_size=STEP_SR,
    overlap_size=OVERLAP_SR, 
    discard_size=DISCARD_SR, 
    output_dir=TMP_OUT_DIR,
    _fill_value=0.0, 
    _format="MEL"
)
#################################

def get_audio_info(filepath):
    """
    Get some properties from  an audio file
    
    SOURCE: https://www.kaggle.com/kneroma/birdclef-mels-computer-public
    """
    with SoundFile(filepath) as f:
        sr = f.samplerate
        frames = f.frames
        duration = float(frames)/sr
    return {"frames": frames, "sr": sr, "duration": duration}

def add_audio_to_df(row):
    """ Wrapper for pandas to add audio info to each row """
    for k,v in get_audio_info(row["f_path"]).items():
        row[k] = v
    return row


def add_clip_weighting(_df):
    def _return_pct_overlap(test_span, clip_span, full_clip_length=160_000):
        upper_lim = min(test_span[-1], clip_span[-1])
        lower_lim = max(test_span[0], clip_span[0])
        return max(0.0, upper_lim-lower_lim)/full_clip_length

    
    __df = _df.copy()
    
    # Create temporary dataframe 
    _tmp_df = pd.DataFrame({"mel_f_path":sorted(glob(os.path.join(TMP_OUT_DIR, "**/*.png")))})
    _tmp_df.insert(0, "test_id", _tmp_df["mel_f_path"].apply(lambda x: x.rsplit("/", 2)[1]))
    _tmp_df.insert(1, "time_span", _tmp_df["mel_f_path"].apply(lambda x: x.rsplit("/", 1)[-1][:-4]))
    _tmp_df.insert(2, "_id_w_timespan", _tmp_df["test_id"]+"/"+_tmp_df["time_span"])
    for i in range(5,65,5):
        _tmp_df[f"clip_{i}_wt"] = _tmp_df.time_span.apply(lambda x: _return_pct_overlap(test_span=((i-5)*SR, i*SR), clip_span=(int(x[:9]), int(x[-9:]))))
    
    return _tmp_df

def get_reduced_df(df, return_indexed_df=True, duration_goal=66_000, verbose=0):
    """ 
    This function willl pare the train dataframe down into a dataframe containing 
    enough examples to add up to a duration of approximately 66,000 seconds of audio.
    
    The restrictions are that we attempt to evenly pull from all birds with a preference
    being given for audio clips that are as close to 60 seconds as possible.
    
    Args:
        df (pd.DataFrame): To be pared down
        return_indexed_df (bool, optional): Whether to return the 
            pared dataframe ... or simply the indices to use for paring.
        duration_goal (int, optional): Sum of durations for pared dataframe
    
    Returns:
        Either a list of indices or a pared down pd.DataFrame
    """
    def _verbose_print(print_duration=True, v_span=100):
        # Verbose printing
        if len(pare_indices)%v_span==0: 
            print(f"\n\n\t{len(pare_indices):>4}-{len(pare_indices)+v_span:<4} {'(DURATION='+str(round(duration, 2))+')' if print_duration else ''}")
        else: 
            print(".", sep="", end="")
            
    pare_indices, duration = [], 0
    redux_df = df.copy()
    redux_df["t_delta"] = (redux_df.duration-60).abs()
    
    _all_species = df.primary_label.unique()
    
    while True:    
        # For loop for species... to be iterated over
        for _species in _all_species:
            # temporary dataframe based on species
            _tmp_df = redux_df[redux_df.primary_label==_species]
            
            # Catch for empty dataframe and verbose print
            if len(_tmp_df)==0: continue
            if verbose: _verbose_print()
            
            pare_indices.append(_tmp_df.sort_values(by="t_delta").index.values[0])
            redux_df=redux_df[~redux_df.index.isin(pare_indices)]                
            
            # Return catch
            duration+=60
            if duration>duration_goal:        
                if return_indexed_df:
                    return df.iloc[pare_indices].reset_index(drop=True)
                else:
                    return pare_indices

def get_melspec(audio, sr, n_mels, fmin, fmax, n_fft, hop_length,):
    
    # Coerce Input
    if type(audio)==str: audio = sf.read(audio, dtype="float32")[0]
    if len(audio.shape)>1: audio=audio[:, 0] 
        
    # Convert To Mel Spectrogram
    melspec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, fmin=fmin, fmax=fmax, n_fft=n_fft, hop_length=hop_length)
    melspec = librosa.power_to_db(melspec).astype(np.float32)
    
    return mono_to_color(melspec)

def mono_to_color(X, eps=1e-6, mean=None, std=None):
    """ https://www.kaggle.com/kneroma/birdclef-mels-computer-public """
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V
                
# We will update segment_audio fn to output/save melspecs instead of ogg files
def segment_audio(audio_path, sample_rate, window_size, 
                  step_size, overlap_size, discard_size, 
                  output_dir, _fill_value=0.0, _format="MEL", 
                  _mspec_kwargs=melspec_kwargs, enforce_len=True,
                  enforce_len_sr=MAX_CLIP_LEN_SR):
    """
    
    Create segments of audio from longer audio clip and save to file
    
    Usage:
        _ = train_df.f_path.progress_apply(lambda x: segment_audio(x, **segment_kwargs))
        
    Args:
        audio_path (str): Path to the audio file to be segmented (.ogg) file
        sample_rate (int): Rate at which the audio was originally sampled (default is 32_000)(Hz)
        window_size (int): Length of desired window segment measured in # of samples.
            i.e. A window segment value of 224_000 would be a 7 second segment assuming
                 a sample rate of 32_000 samples per second.
        step_size (int): Length of desired step between segments measured in # of samples.
            i.e. A step size value of 149_333 would be approximately a 4.6667 second step
                 assuming a sample rate of 32_000 samples per second.
        overlap_size (int): Length of overlap between consecutive segments measured in # of samples.
            i.e. An overlap value of 74_667 would be approximately a 2.33333 second segment 
                assuming a sample rate of 32_000 samples per second.
        discard_size (int): Length of empty audio identifing a segment to be discarded measured in # of samples.
            i.e. A discard size value of 192_000 would indicate that segments containing more than 6
                 seconds of empty audio should be discarded... assuming a sample rate of 32_000.
        output_dir (str): The path to the directory to output the segmented audio files
        _mspec_kwargs (dict): Keyword arguments for mel spectrogram creation
        _fill_value (float, optional): The 'empty' audio value to pad the original audio with 
            to allow the original audio to be segmented evenly without having a single shorter/longer
            segment at the end/beginning.
        _format (str, optional):
            The format for which to save the segments
        enforce_len (bool, optional):
            Whether to limit the audio clip read to the maximum duratino (60 seconds usually)
        enforce_len_sr (int, optional):
            Only used if enforce_len==True. Number of frames of audio to limit audio file to.
    
    Returns:
        None; This function will simply save audio segments to file. No actual entities
              will be returned in the interpreter.
    
    """
    # Prepare
    example_id = audio_path.rsplit("/", 1)[-1][:-4]
    save_dir = os.path.join(output_dir, example_id)
    if not os.path.isdir(save_dir): os.makedirs(save_dir, exist_ok=True)
        
    # Process
    for i, audio_block in enumerate(sf.blocks(audio_path, window_size, overlap_size, frames=enforce_len_sr if enforce_len else -1, dtype="float32", fill_value=_fill_value)):
        
        # Discard if not the first clip and less than the discard amount of the clip is padding
        if enforce_len or i==0 or np.count_nonzero(audio_block==0.0)<discard_size:
            
            # Get start and end slice locations in samples
            _start = step_size*i
            _end = _start+window_size
            
            if _format.lower()=="ogg":
                # Save the file
                sf.write(os.path.join(save_dir, f"{_start:>09}_{_end:>09}.ogg"), audio_block, samplerate=sample_rate, format="OGG")
            elif _format.lower()=="mel":
                cv2.imwrite(os.path.join(save_dir, f"{_start:>09}_{_end:>09}.png"), get_melspec(audio_block, **_mspec_kwargs))            
                
def mp_segment_audio(_df, _backend="loky", _njobs=-1, _prefer="processes", _verbose=0, _enforce_len=True):
    """ Parallel file creation of mel spectrograms
        
    Args:
        _df (pd.DataFrame): The dataframe to use as base for file creation
        _backend (str, optional): Joblib backend
            NOTE: `loky` backend operates almost twice as fast as others 
                  so it's recommended to leave it be... for default kaggle instance
        _njobs (int, optional ): Number of concurrent workers for joblib
        _prefer (str, optional): Whether to prefer proccesses or threads in joblib
        _verbose (int, optional): Whether to let joblib display progress
    
    Returns:
        None; Simply call the function that will write files to disk and let it happen
    """
    _ogg_file_paths = list(set(_df.ogg_f_path.tolist()))
    n_test_files = len(_ogg_file_paths)
    
    _ = Parallel(n_jobs=_njobs, backend=_backend, prefer=_prefer, verbose=_verbose)(
            delayed(segment_audio)(audio_path=_f_path, **segment_kwargs, _mspec_kwargs=melspec_kwargs, enforce_len=_enforce_len) for _f_path in _ogg_file_paths
        )
    
def augment_batch(img_batch, pp_fn):
    """ Simple pass through for now...
    
    Investigate this 
        --> https://www.tensorflow.org/io/tutorials/audio#trim_the_noise
    
    """
    return pp_fn(tf.cast(img_batch, tf.float32))

def get_ss_row_id_base(tensor_str):
    return tf.strings.split(tensor_str, sep="/", maxsplit=-1)[-2]

def get_timespan(tensor_str):
    return tf.strings.split(tf.strings.split(tensor_str, sep="/", maxsplit=-1)[-1], sep=".")[0]

def tf_load_image(img_path, reshape_to=(128,281,3)):
    return tf.reshape(tf.image.decode_png(tf.io.read_file(img_path), channels=3), reshape_to)

def get_ds_for_inference(_clip_df, _bs, _pp, do_plot=False):
    _mel_files = _clip_df.mel_f_path.tolist()
    n_files = len(_mel_files)

    # Create Filepath Dataset
    _ds = tf.data.Dataset.from_tensor_slices(_mel_files)

    # Read The File From Path and Get Relevant Info
    _ds = _ds.map(lambda path: (tf_load_image(path), get_ss_row_id_base(path), get_timespan(path)), num_parallel_calls=tf.data.AUTOTUNE)
    
    # Optional Plot
    if do_plot:
        for __img, _, _ in _ds.take(1):
            plt.imshow(__img[..., 0])
            plt.title("EXAMPLE SPECTROGRAM", fontweight="bold")
            plt.show()

    # Pipeline preparing dataset for inference
    _ds = _ds.batch(_bs)\
             .map(lambda img, row_id, t_span: (augment_batch(img, _pp), row_id, t_span), num_parallel_calls=tf.data.AUTOTUNE)\
             .prefetch(tf.data.AUTOTUNE)

    return _ds, n_files

def get_pred_df(_df,
                _batch_size=192, 
                _mlbl_fpath="../input/training-birdclef-22-multilabel-nocall/f1_effnet_b0_128x281x3_cwt_10__doublesoftloss",
                _ncall_fpath="/kaggle/input/birdclef-2022-nocall-effnetb0/effnet_b0_callnocall__7S_128x281x3",
                _pp_fn=tf.keras.applications.efficientnet.preprocess_input):

    # Load the models
    nc_model = tf.keras.models.load_model(_ncall_fpath, compile=False)
    mlbl_model = tf.keras.models.load_model(_mlbl_fpath, compile=False)

    # Get the dataset
    ds, n_ex = get_ds_for_inference(_df, _batch_size, _pp_fn, do_plot=True)

    # Inference Loop
    for i, (_img_batch, _row_id_batch, _timespan_batch) in tqdm(enumerate(ds), total=(n_ex//_batch_size+1)):
        if i==0:
            # I WAS using .predict() ... 
            # however this leads to memory leakage and the resulting submission error
            mlbl_preds = mlbl_model(_img_batch, training=False)
            nc_preds = nc_model(_img_batch, training=False)
            ss_row_id_bases  = _row_id_batch
            timespans = _timespan_batch
        else:
            # I WAS using .predict() ... but this leads to memory leakage and the resulting submission error
            # however this leads to memory leakage and the resulting submission error
            mlbl_preds = tf.concat([mlbl_preds, mlbl_model(_img_batch, training=False)], axis=0)
            nc_preds = tf.concat([nc_preds, nc_model(_img_batch, training=False)], axis=0)
            ss_row_id_bases = tf.concat([ss_row_id_bases, _row_id_batch], axis=0)
            timespans = tf.concat([timespans, _timespan_batch], axis=0)

    pred_df = pd.DataFrame({"_id_w_timespan":[a+"/"+b for a,b in zip(
        [x.decode() for x in ss_row_id_bases.numpy()], [x.decode() for x in timespans.numpy()]
    )]})

    pred_df = pred_df.join(pd.DataFrame(mlbl_preds.numpy()[:, np.array(SCORED_BIRD_INTS)], columns=SCORED_BIRD_STRS))
    pred_df["call_conf"] = nc_preds.numpy()[:, 1]
    _df = _df.merge(pred_df, on="_id_w_timespan")
    return _df

def get_preds_per_5sec(_df, base_row_id, to_bool=False,
                       do_scored=True, scored_thresh=0.1, 
                       do_pwr=False, pwr=20, 
                       do_nocall=False, nocall_thresh=0.5, 
                       do_combo=False, combo_thresh=0.9):
    """ TBD """
    
    # Initialize
    _sub_df = _df[_df.test_id==base_row_id]
    sub_pred_map = {"row_id":[], "target":[]}
    
    # Iterate over clip segments
    for i in range(5, 65, 5):
        # Get sub dataframe
        _span_sub_df = _sub_df[_sub_df[f"clip_{i}_wt"]!=0.0]
        
        # Get model predictions
        average_call_conf = (_span_sub_df.call_conf*_span_sub_df[f"clip_{i}_wt"]).sum()/_span_sub_df[f"clip_{i}_wt"].sum()
        average_preds = np.dot(_span_sub_df[f"clip_{i}_wt"], _span_sub_df[SCORED_BIRD_STRS].to_numpy())/_span_sub_df[f"clip_{i}_wt"].sum()
        
        if do_pwr: average_preds = average_preds**pwr
        
        if to_bool:
            if do_combo:
                thresh_preds = np.where(average_preds*average_call_conf>combo_thresh, True, False)
            elif do_nocall:
                if average_call_conf>nocall_thresh:
                    thresh_preds = np.where(average_preds>scored_thresh, True, False)
                else:
                    thresh_preds = [False,]*len(SCORED_BIRD_STRS)
            else:
                thresh_preds = np.where(average_preds>scored_thresh, True, False)
        else:
            if do_combo:
                thresh_preds = average_preds*average_call_conf
            else:
                thresh_preds = average_preds
            
        for _bird, _pred in zip(SCORED_BIRD_STRS, thresh_preds):
            sub_pred_map["row_id"].append(base_row_id+f"_{_bird}_{i}")
            sub_pred_map["target"].append(_pred)
    
    return pd.DataFrame(sub_pred_map)

# def get_preds_per_5sec(_df, base_row_id, 
#                        do_scored=True, scored_thresh=0.1, 
#                        do_pwr=False, pwr=10, 
#                        do_nocall=False, nocall_thresh=0.1, 
#                        do_combo=False, combo_thresh=0.9, take_top_only=False):
#     _sub_df = _df[_df.test_id==base_row_id]
#     sub_pred_map = {"row_id":[], "target":[]}
    
#     for i in range(5, 65, 5):
#         _span_sub_df = _sub_df[_sub_df[f"clip_{i}_wt"]!=0.0]
#         average_call_conf = (_span_sub_df.call_conf*_span_sub_df[f"clip_{i}_wt"]).sum()/_span_sub_df[f"clip_{i}_wt"].sum()
#         average_preds = np.dot(_span_sub_df[f"clip_{i}_wt"], _span_sub_df[SCORED_BIRD_STRS].to_numpy())/_span_sub_df[f"clip_{i}_wt"].sum()
        
#         if not take_top_only:
#             if do_pwr:
#                 average_preds = average_preds**pwr
#             if combo_thresh:
#                 thresh_preds = np.where(average_preds*average_call_conf>combo_thresh, True, False)
#             elif do_nocall:
#                 if average_call_conf>nocall_thresh:
#                     thresh_preds = np.where(average_preds>scored_thresh, True, False)
#                 else:
#                     thresh_preds = [False,]*len(SCORED_BIRD_STRS)
#             else:
#                 thresh_preds = np.where(average_preds>scored_thresh, True, False)
#         else:
#             if (average_call_conf>nocall_thresh) and (average_preds.max()>0.2):
#                 max_conf = average_preds.max()
#                 min_yes = max_conf-(max_conf**2)*0.25
#                 if i==60:
#                     if min_yes<0.25:
#                         print(min_yes)
#                 thresh_preds = np.where(average_preds>=min_yes, True, False)
#             else:
#                 thresh_preds = [False,]*len(SCORED_BIRD_STRS)
                
#         for _bird, _pred in zip(SCORED_BIRD_STRS, thresh_preds):
#             sub_pred_map["row_id"].append(base_row_id+f"_{_bird}_{i}")
#             sub_pred_map["target"].append(_pred)
#     return pd.DataFrame(sub_pred_map)

In [ ]:
# Load dataframe
train_df = pd.read_csv(os.path.join(DATA_DIR, "train_metadata.csv"))

# Add filepath to training dataframe
train_df["f_path"] = train_df.filename.progress_apply(lambda x: DATA_DIR+"/train_audio/"+x)

# Initialize Some Constants/Details
str2int_lbl_map = {k:i for i, k in enumerate(sorted(list(train_df.primary_label.unique())))}
int2str_lbl_map = {v:k for k,v in str2int_lbl_map.items()}
N_CLASSES = len(str2int_lbl_map)
with open('../input/birdclef-2022/scored_birds.json', 'r') as f: SCORED_BIRD_STRS=json.load(f)
SCORED_BIRD_INTS = [str2int_lbl_map[x] for x in SCORED_BIRD_STRS]

# Add duration information to training dataframe
train_df = train_df.progress_apply(add_audio_to_df, axis=1)

# Reduce dataframe while maintaining integrity of class distribution (targeting 60 second clips)
#
# NOTE: The 249.048 clip is to capture one hawaiian bird that only has one training example
train_df = train_df[((train_df.duration>3) & (train_df.duration<180)) | (train_df.duration==249.048)].reset_index(drop=True)

# This samples the reduced dataframe to further reduce it 
# so that the total duration of samples is 1/5th of the test set
#
# NOTE: This will offer preference to clips that are close to 60 seconds
#       but will cycle through species while doing so (so some species with
#       clips that are not near 60 seconds may have more extreme clip durations)
# NOTE: duration_goal of 6_600 is 1/50th, 33_000 is 1/10th and 66_000 is 1/5th.
train_df = get_reduced_df(train_df, return_indexed_df=True, verbose=0, duration_goal=66_000)

# Create a new dataframe that mimics the layout of the test dataframe
train_for_test_df = train_df[["filename", "f_path"]]
train_for_test_df.columns=["file_id", "ogg_f_path"]
train_for_test_df.file_id = train_for_test_df.file_id.apply(lambda x: x[:-4])

# Do some printing
print(f"\n... REDUCED TRAIN DATAFRAME - {len(train_df)} ROWS ...\n")
display(train_df.head(3))

# Save files to disk... [small difference here]
mp_segment_audio(train_for_test_df, _verbose=1)

# Add weights that show the overlap between the 7-second segment
# and the 5-second scoring window
clip_df = add_clip_weighting(train_for_test_df)

# Get the prediction dataframe
pred_df = get_pred_df(clip_df,)

# Get `sample_submission.csv` format inference dataframe
viz_ss_df = pd.concat([
    get_preds_per_5sec(pred_df, _test_id, nocall_thresh=0.1, do_nocall=True, scored_thresh=0.025, ) \
    for _test_id in tqdm(pred_df.test_id.unique(), 
                         total=len(pred_df.test_id.unique()))
]).reset_index(drop=True)
display(viz_ss_df)

<br>

<a id="viz_fn"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #34558b; background-color: #ffffff;" id="viz_fn">2&nbsp;&nbsp;VISUALIZATION FUNCTIONALITY&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


In [ ]:
def get_sub_pred_df(_pred_df, _row_id=None, return_row_id=False):
    """ Ease of use fn 
    
    If _row_id is none a random example is pulled
    """
    if _row_id is None:
        _row_id = np.random.choice(viz_ss_df.row_id.apply(lambda x: x.split("_", 1)[0]).unique())
    _sub_pred_df = _pred_df.copy()[_pred_df.row_id.str.contains(_row_id)].reset_index(drop=True)
    
    if return_row_id:
        return _sub_pred_df, _row_id
    else:
        return _sub_pred_df

def plot_clip_preds(sub_pred_df, 
                    row_identifier="Not Specified",
                    label_grid_values=True,
                    n_segments=12, 
                    _cmap="magma",
                    _fig_size=(20,13),
                    _xlabel="5 Second Segments",
                    _ylabel="Species Label (Identifier)",
                    n_float_decimal_places=3,):
    """ TBD """
    # Plotting Helpers
    _x_start, _x_end, _y_start, _y_end = 3.0, 9.0, 6.0, 12.0
    _extent = [_x_start, _x_end, _y_start, _y_end]
    n_species = len(SCORED_BIRD_STRS)
    
    # For later
    float_style = sub_pred_df["target"][0].dtype=="float"
    
    # Get predictions as numpy array (n_segments, n_bird_species)
    pred_arr = sub_pred_df["target"].to_numpy().reshape(-1, n_species).T

    # Make figure
    plt.figure(figsize=_fig_size)
    plt.xlabel(_xlabel, fontweight="bold")
    plt.ylabel(_ylabel, fontweight="bold")
    plt.title(f"Audio Clip Predictions - Visualization - {row_identifier}", fontweight="bold")
    
    # The x and y axis ticks and labels
    plt.xticks(ticks=np.linspace(_x_start, 
                                 _x_end, 
                                 n_segments+1), 
               labels=[str(x) for x in np.arange(0, 65, 5)], fontweight="bold")
    plt.yticks(ticks=np.linspace(_y_start+6/(n_species+1)/2, 
                                 _y_end-(6/(n_species+1))/2, 
                                 n_species), 
               labels=SCORED_BIRD_STRS, fontweight="bold")
    
    # Do the grid numbering if desired
    if label_grid_values:
        _jump_x = (_x_end-_x_start)/(2.0*n_segments)
        _jump_y = (_y_end-_y_start)/(2.0*n_species)
        _x_positions = np.linspace(start=_x_start, stop=_x_end, num=n_segments, endpoint=False)
        _y_positions = np.linspace(start=_y_start, stop=_y_end, num=n_species, endpoint=False)
        for y_index, y in enumerate(_y_positions):
            for x_index, x in enumerate(_x_positions):
                _label = pred_arr[y_index, x_index]
                _text_x = x+_jump_x
                _text_y = y+_jump_y
                if float_style: _label=round(_label, n_float_decimal_places)
                plt.text(_text_x, _text_y, _label, color='black' if _label>(pred_arr.max()*0.4) else 'white', ha='center', va='center')

    # Display
    plt.imshow(pred_arr, cmap=_cmap, extent=_extent, origin='lower', interpolation='None')
    
    if float_style:
        plt.colorbar()
        plt.clim(0.0, 1.0)
        
    plt.tight_layout()
    plt.show()

**Below Is The Macro F1 Score**

We know a few things about the test set and we have endeavoured to make our train set similar to this
* Only 60 second clips
* If we submit all False (no-call) then we should score 0.45-0.50
* All non-scored birds are no calls

Here is some information about how we determine some of the ground truth data.
* If no scored-bird is found in the primary or secondary labels column than all segments from that audio clip are considered to be False for all scored birds
* If a scored-bird is found in the primary column than we naively assume that all segments are that bird
* If a scored-bird is found in the secondary column we will assume it is not present (this is erroneous ... but we can't help it right now).

In [ ]:
filename_to_primary = train_df.groupby("filename").primary_label.first().to_dict()
filename_to_primary = {k.split("/")[-1][:-4]:v for k,v in filename_to_primary.items()}

def get_gt_lbl(row_id):
    xc_id, _species, _seg = row_id.split("_")
    gt_primary = filename_to_primary[xc_id]
    if gt_primary==_species:
        return 1.0
    else:
        return 0.0
        
viz_ss_df["gt_lbl"] = viz_ss_df.row_id.progress_apply(get_gt_lbl)
viz_ss_df

In [ ]:
def comp_metric(y_true, y_pred, epsilon=1e-9):
    """ Function to calculate competition metric in an sklearn like fashion

    Args:
        y_true{array-like, sparse matrix} of shape (n_samples, n_outputs)
            - Ground truth (correct) target values.
        y_pred{array-like, sparse matrix} of shape (n_samples, n_outputs)
            - Estimated targets as returned by a classifier.
    Returns:
        The single calculated score representative of this competitions evaluation
    """
    
    # Get representative confusion matrices for each label
    mlbl_cms = sklearn.metrics.multilabel_confusion_matrix(y_true, y_pred)

    # Get two scores (TP and TN SCORES)
    tp_scores = np.array([
        mlbl_cm[1, 1]/(epsilon+mlbl_cm[:, 1].sum()) \
        for mlbl_cm in mlbl_cms
        ])
    
    tn_scores = np.array([
        mlbl_cm[0, 0]/(epsilon+mlbl_cm[:, 0].sum()) \
        for mlbl_cm in mlbl_cms
        ])

    # Get average
    tp_mean = tp_scores.mean()
    tn_mean = tn_scores.mean()

    return round((tp_mean+tn_mean)/2, 8)

def get_species_arrs(pred_y, true_y, thresh=0.5, dynamic_thresh=True):
    """ Get species-wise categorical binary predictions """
    # Prevent problems
    if thresh==0.0: thresh+=1e-9
    
    n_species = len(SCORED_BIRD_STRS)
    n_ex = (len(pred_y)//n_species)
    pred_arr = np.zeros((n_ex, n_species+1), dtype=np.uint8)
    gt_arr = np.zeros((n_ex, n_species+1), dtype=np.uint8)
    
    for i, idx in enumerate(range(0, len(pred_y), n_species)):
        max_conf = average_preds.max()
        min_yes = max_conf-(max_conf**2)*0.25
        thresh_preds = np.where(average_preds>=min_yes, True, False)
        
        pred_arr[i, :n_species] = np.where(pred_y[idx:(idx+n_species)]>=thresh, 1, 0)
        gt_arr[i, :n_species] = np.where(true_y[idx:(idx+n_species)]>=thresh, 1, 0)
        pred_arr[i, n_species]=(1 if pred_arr[i].sum()==0 else 0)
        gt_arr[i, n_species]=(1 if gt_arr[i].sum()==0 else 0)
        
    return pred_arr, gt_arr      


thresholds   = []
f1_scores    = []
comp_scores  = []
# for _thresh in tqdm(np.arange(0, 1.005, 0.05), total=len(np.arange(0, 1, 0.005))+1):
pred_sps_arrs, gt_sps_arrs = get_species_arrs(viz_ss_df.target.to_numpy(), viz_ss_df.gt_lbl.to_numpy(), thresh=0.5)
_f1_score = sklearn.metrics.f1_score(gt_sps_arrs[:, :-1], pred_sps_arrs[:, :-1], average='macro')
_comp_score = comp_metric(gt_sps_arrs[:, :-1], pred_sps_arrs[:, :-1])

# thresholds.append(_thresh)
# f1_scores.append(_f1_score)
# comp_scores.append(_comp_score)

print(f"\n\n\nSTARTING TEST w/ THRESH={_thresh}%\n")
print(f"\n\t--> MACRO F1 SCORE: {_f1_score}")
print(f"\t--> CLASSWISE COMPETITION METRIC ACCURACY: {_comp_score}\n")

In [ ]:
fig = px.line(x=thresholds, y=[f1_scores, comp_scores], 
              labels={"wide_variable_0":"<b>F1 Score</b>", "wide_variable_1":"<b>Competition Metric</b>"},
              range_x=(0.0, 1.0), range_y=(0.0, 1.0), 
              title="<b>Threshold v. Model Performance (F1 & Comp)</b>",
             )
fig.show()

In [ ]:
# TOP COMP SCORE - THRESH=0.9874
# LAST STABLE    - THRESH=0.95-0.96

pred_sps_arrs, gt_sps_arrs = get_species_arrs(viz_ss_df.target.to_numpy(), viz_ss_df.gt_lbl.to_numpy(), thresh=0.95)
print(sklearn.metrics.f1_score(gt_sps_arrs[:, :-1], pred_sps_arrs[:, :-1], average='macro'))
print(comp_metric(gt_sps_arrs[:, :-1], pred_sps_arrs[:, :-1]))

In [ ]:
print(pred_sps_arrs.sum(axis=0))
print()
print(gt_sps_arrs.sum(axis=0))

In [ ]:
SCORED_BIRD_STRS[11]

In [ ]:
print(pred_sps_arrs.sum(axis=0))
print()
print(gt_sps_arrs.sum(axis=0))

In [ ]:
def comp_metric(y_true, y_pred, epsilon=1e-9):
    """ Function to calculate competition metric in an sklearn like fashion

    Args:
        y_true{array-like, sparse matrix} of shape (n_samples, n_outputs)
            - Ground truth (correct) target values.
        y_pred{array-like, sparse matrix} of shape (n_samples, n_outputs)
            - Estimated targets as returned by a classifier.
    Returns:
        The single calculated score representative of this competitions evaluation
    """
    
    # Get representative confusion matrices for each label
    mlbl_cms = sklearn.metrics.multilabel_confusion_matrix(y_true, y_pred)

    # Get two scores (TP and TN SCORES)
    tp_scores = np.array([
        mlbl_cm[1, 1]/(epsilon+mlbl_cm[:, 1].sum()) \
        for mlbl_cm in mlbl_cms
        ])
    
    tn_scores = np.array([
        mlbl_cm[0, 0]/(epsilon+mlbl_cm[:, 0].sum()) \
        for mlbl_cm in mlbl_cms
        ])

    print(tp_scores)
    print(tn_scores)
    
    # Get average
    tp_mean = tp_scores.mean()
    tn_mean = tn_scores.mean()

    return round((tp_mean+tn_mean)/2, 8)

print(comp_metric(gt_sps_arrs[:, :-1], pred_sps_arrs[:, :-1]))

In [ ]:
print(f"\n\n\nSTARTING TEST w/ THRESH={_thresh}%\n")
pred_sps_arrs, gt_sps_arrs = get_species_arrs(viz_ss_df.target.to_numpy(), viz_ss_df.gt_lbl.to_numpy(), thresh=0.99647)

print(f"\n\t--> MACRO F1 SCORE: {sklearn.metrics.f1_score(gt_sps_arrs[:, -1], pred_sps_arrs[:, -1], average='macro')}")
print(f"\t--> CLASSWISE COMPETITION METRIC ACCURACY: {comp_metric(gt_sps_arrs[:, -1], pred_sps_arrs[:, -1])}\n")

In [ ]:
for _thresh in tqdm(range(950, 1005, 5), total=10):
    print(f"\n\n\nSTARTING TEST w/ THRESH={_thresh}%\n")
    pred_sps_arrs, gt_sps_arrs = get_species_arrs(viz_ss_df.target.to_numpy(), viz_ss_df.gt_lbl.to_numpy(), thresh=_thresh/1000)

    print(f"\n\t--> MACRO F1 SCORE: {sklearn.metrics.f1_score(gt_sps_arrs[:, -1], pred_sps_arrs[:, -1], average='macro')}")
    print(f"\t--> CLASSWISE COMPETITION METRIC ACCURACY: {comp_metric(gt_sps_arrs[:, -1], pred_sps_arrs[:, -1])}\n")

**Below are some examples of scored species**
* Notice that even in padded audio my model still predicts with some confidence... not good.

In [ ]:
for _, _row in train_df[train_df.primary_label.isin(SCORED_BIRD_STRS)].groupby("primary_label").sample(1).reset_index(drop=True).iterrows():
    xc_id = _row["filename"].split("/")[-1][:-4]
    print(f"\n\n\tRANDOM EXAMPLE FOR SCORED SPECIES --> {_row['primary_label'].upper()} ({xc_id}) \n")
    display(ipd.Audio(_row.f_path))
    display(pd.DataFrame(_row).T)
    
    sub_viz_df = get_sub_pred_df(viz_ss_df, _row_id=xc_id)
    plot_clip_preds(sub_viz_df, xc_id)

**Below are some examples of non-scored species**

* As you can see my model is very prone to false-positives...

In [ ]:
for i, (_, _row) in enumerate(train_df[~train_df.primary_label.isin(SCORED_BIRD_STRS)].groupby("primary_label").sample(1).reset_index(drop=True).iterrows()):
    if i==10: break
    xc_id = _row["filename"].split("/")[-1][:-4]
    print(f"\n\n\tRANDOM EXAMPLE FOR NON-SCORED SPECIES --> {_row['primary_label'].upper()} ({xc_id}) \n")
    display(ipd.Audio(_row.f_path))
    display(pd.DataFrame(_row).T)
    sub_viz_df = get_sub_pred_df(viz_ss_df, _row_id=xc_id)
    plot_clip_preds(sub_viz_df, xc_id)

**Here's an example comparing the predictions for a clip containing a Canada Goose vs. a Hawaiian Goose**
* This is just a simple example of the type of comparison you can do... obviously the functionality can be expanded

In [ ]:
# ex_can_goose = "XC609624"
# ex_haw_goose = "XC326899"

# _row = train_df[train_df.filename.str.contains(ex_can_goose)].reset_index(drop=True).iloc[0]
# xc_id = _row["filename"].split("/")[-1][:-4]
# print(f"\n\n\tCANADA GOOSE EXAMPLE --> {_row['primary_label'].upper()} ({xc_id}) \n")
# display(ipd.Audio(_row.f_path))
# display(pd.DataFrame(_row).T)
# sub_viz_df = get_sub_pred_df(viz_ss_df, _row_id=xc_id)
# plot_clip_preds(sub_viz_df, xc_id)

# _row = train_df[train_df.filename.str.contains(ex_haw_goose)].reset_index(drop=True).iloc[0]
# xc_id = _row["filename"].split("/")[-1][:-4]
# print(f"\n\n\tHAWAIIAN GOOSE EXAMPLE --> {_row['primary_label'].upper()} ({xc_id}) \n")
# display(ipd.Audio(_row.f_path))
# display(pd.DataFrame(_row).T)
# sub_viz_df = get_sub_pred_df(viz_ss_df, _row_id=xc_id)
# plot_clip_preds(sub_viz_df, xc_id)

**Bool Version Of Visualization**

In [ ]:
BOOL_THRESH = 0.75
viz_ss_df.target = viz_ss_df.target>BOOL_THRESH
viz_ss_df

In [ ]:
for _, _row in train_df[train_df.primary_label.isin(SCORED_BIRD_STRS)].groupby("primary_label").sample(1).reset_index(drop=True).iterrows():
    xc_id = _row["filename"].split("/")[-1][:-4]
    print(f"\n\n\tRANDOM EXAMPLE FOR SCORED SPECIES --> {_row['primary_label'].upper()} ({xc_id}) \n")
    display(ipd.Audio(_row.f_path))
    display(pd.DataFrame(_row).T)
    sub_viz_df = get_sub_pred_df(viz_ss_df, _row_id=xc_id)
    plot_clip_preds(sub_viz_df, xc_id)